In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.6/779.6 kB 10.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-

# **Object Tracking**


In [7]:
!yolo track source="/content/final_traffic.mp4" save=True

WARNING ⚠️ 'model' argument is missing. Using default 'model=yolov8n.pt'.
Ultralytics YOLOv8.2.28 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (Intel Xeon 2.20GHz)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

video 1/1 (frame 1/527) /content/final_traffic.mp4: 384x640 3 cars, 349.1ms
video 1/1 (frame 2/527) /content/final_traffic.mp4: 384x640 3 cars, 219.1ms
video 1/1 (frame 3/527) /content/final_traffic.mp4: 384x640 3 cars, 208.1ms
video 1/1 (frame 4/527) /content/final_traffic.mp4: 384x640 4 cars, 190.3ms
video 1/1 (frame 5/527) /content/final_traffic.mp4: 384x640 4 cars, 205.8ms
video 1/1 (frame 6/527) /content/final_traffic.mp4: 384x640 4 cars, 207.1ms
video 1/1 (frame 7/527) /content/final_traffic.mp4: 384x640 4 cars, 198.6ms
video 1/1 (frame 8/527) /content/final_traffic.mp4: 384x640 4 cars, 1 bus, 200.3ms
video 1/1 (frame 9/527) /content/final_traffic.mp4: 384x640 4 cars, 1 bus, 211.6ms
video 1/1 (frame 10/527) /content/final_traffic.mp4: 384x640 4

In [8]:
import cv2
import numpy as np
from ultralytics import YOLO

from ultralytics.utils.checks import check_imshow
from ultralytics.utils.plotting import Annotator, colors

from collections import defaultdict

In [9]:
track_history = defaultdict(lambda: [])
model = YOLO("yolov8n.pt")
names = model.model.names

In [27]:
video_path = "/content/final_traffic.mp4"
cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), "Error reading video file"

In [ ]:
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

result = cv2.VideoWriter("object_tracking.avi",
                       cv2.VideoWriter_fourcc(*'mp4v'),
                       fps,
                       (w, h))

In [ ]:

while cap.isOpened():
    success, frame = cap.read()
    if success:
        results = model.track(frame, persist=True, verbose=False)
        boxes = results[0].boxes.xyxy.cpu()

        if results[0].boxes.id is not None:

            # Extract prediction results
            clss = results[0].boxes.cls.cpu().tolist()
            track_ids = results[0].boxes.id.int().cpu().tolist()
            confs = results[0].boxes.conf.float().cpu().tolist()

            # Annotator Init
            annotator = Annotator(frame, line_width=2)

            for box, cls, track_id in zip(boxes, clss, track_ids):
                annotator.box_label(box, color=colors(int(cls), True), label=names[int(cls)])

                # Store tracking history
                track = track_history[track_id]
                track.append((int((box[0] + box[2]) / 2), int((box[1] + box[3]) / 2)))
                if len(track) > 30:
                    track.pop(0)

                # Plot tracks
                points = np.array(track, dtype=np.int32).reshape((-1, 1, 2))
                cv2.circle(frame, (track[-1]), 7, colors(int(cls), True), -1)
                cv2.polylines(frame, [points], isClosed=False, color=colors(int(cls), True), thickness=2)

        result.write(frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break

result.release()
cap.release()
cv2.destroyAllWindows()

# **Vechile counting**



In [41]:
! git clone https://github.com/yourusername/traffic-object-counting.git

Cloning into 'traffic-object-counting'...
fatal: could not read Username for 'https://github.com': No such device or address


In [33]:
from ultralytics import YOLO
from ultralytics.solutions import object_counter
import cv2

In [36]:
model = YOLO("yolov8n.pt")
cap = cv2.VideoCapture("/content/traffic.mp4")
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

In [37]:
line_points = [(402, 210), (377, 284)]

In [38]:
video_writer = cv2.VideoWriter("object_output.mp4",
                       cv2.VideoWriter_fourcc(*'mp4v'),
                       fps,
                       (w, h))

In [39]:
counter = object_counter.ObjectCounter(classes_names=model.names)
args = {
    'view_img': True,
    'reg_pts': line_points,
    'classes_names': model.names,
    'draw_tracks': True,
    'line_thickness': 2
}


WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Line Counter Initiated.


In [40]:
while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break
    tracks = model.track(im0, persist=True, show=False)

    im0 = counter.start_counting(im0, tracks)
    video_writer.write(im0)

cap.release()
video_writer.release()
cv2.destroyAllWindows()


0: 384x640 10 cars, 169.3ms
Speed: 3.0ms preprocess, 169.3ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 133.2ms
Speed: 3.9ms preprocess, 133.2ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 136.8ms
Speed: 3.9ms preprocess, 136.8ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 127.7ms
Speed: 3.9ms preprocess, 127.7ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 123.9ms
Speed: 3.9ms preprocess, 123.9ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 136.5ms
Speed: 5.4ms preprocess, 136.5ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 137.1ms
Speed: 4.3ms preprocess, 137.1ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 1 truck, 127.7ms
Speed: 4.2ms preprocess, 127.7ms inference, 1.3ms postprocess per image a

In [ ]:
import cv2
from ultralytics import YOLO, solutions

model = YOLO("yolov8n.pt")
cap = cv2.VideoCapture("/content/traffic.mp4")
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Define line points
line_points = [(20, 617), (1080, 617)]


# Video writer
video_writer = cv2.VideoWriter("object_counting.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# Init Object Counter
counter = solutions.ObjectCounter(
    view_img=True,
    reg_pts=line_points,
    classes_names=model.names,
    draw_tracks=True,
    line_thickness=2,
)

while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break
    tracks = model.track(im0, persist=True, show=False)

    im0 = counter.start_counting(im0, tracks)
    video_writer.write(im0)

cap.release()
video_writer.release()
cv2.destroyAllWindows()

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

Line Counter Initiated.

0: 384x640 10 cars, 197.7ms
Speed: 3.9ms preprocess, 197.7ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 149.8ms
Speed: 4.4ms preprocess, 149.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 154.7ms
Speed: 6.2ms preprocess, 154.7ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 148.5ms
Speed: 5.0ms preprocess, 148.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 147.6ms
Speed: 4.6ms preprocess, 147.6ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 148.7ms
Speed: 4.0ms preprocess, 148.7ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 156.5ms
Speed: 5.7ms preprocess, 156.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1